<a href="https://colab.research.google.com/github/deterministic-algorithms-lab/PunctFormer/blob/main/notebooks/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prep

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!wget http://opus.nlpl.eu/download.php?f=WMT-News/v2019/mono/en.txt.gz -O en.txt.gz
!gzip -d en.txt.gz

--2020-12-19 17:32:14--  http://opus.nlpl.eu/download.php?f=WMT-News/v2019/mono/en.txt.gz
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-WMT-News/v2019/mono/en.txt.gz [following]
--2020-12-19 17:32:14--  https://object.pouta.csc.fi/OPUS-WMT-News/v2019/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133632 (130K) [application/gzip]
Saving to: ‘en.txt.gz’

en.txt.gz           100%[===================>] 130.50K   526KB/s    in 0.2s    

2020-12-19 17:32:16 (526 KB/s) - ‘en.txt.gz’ saved [133632/133632]



In [ ]:
import gc
import string
from tqdm.auto import tqdm

import pandas as pd

In [ ]:
punctuations = "?!,-:;."

In [ ]:
with open("/content/en.txt") as fp:
    lines = fp.readlines()

In [ ]:
def cleanString(strval):
    return "".join(" " if i in punctuations else i for i in strval.lower().strip(punctuations))

In [ ]:
dest = []
for i, line in enumerate(lines):
    lines[i] = line.replace("\n", "")
    s = " ".join(cleanString(i) for i in line.split())
    dest.append(s)

In [ ]:
df = pd.DataFrame({"input_text":dest, "target_text":lines})

In [ ]:
!mkdir data seq2seq punctuator
with open("/content/data/test.source", 'w') as fp_src:
    with open("/content/data/test.target", 'w') as fp_tgt:
        for ix in df.index:
            src_txt = df["input_text"][ix]
            dest_txt = df["target_text"][ix]
            #skipping empty strings
            if src_txt != "":
                fp_src.write("{}\n".format(src_txt))
                fp_tgt.write("{}\n".format(dest_txt))

gc.collect()

90

## Transformer

In [ ]:
from transformers import pipeline, AutoTokenizer, BartForConditionalGeneration, T5ForConditionalGeneration

In [ ]:
model = BartForConditionalGeneration.from_pretrained("/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap", max_length=50000)

tokenizer = AutoTokenizer.from_pretrained("/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap")
restore = pipeline('translation_xx_to_yy', model=model.to("cuda"), tokenizer=tokenizer, device=0)

In [ ]:
%%time
text = "hello how are you"

print(restore(text)[0]["translation_text"])

Hello, how are you?
CPU times: user 95.2 ms, sys: 16.6 ms, total: 112 ms
Wall time: 318 ms


In [ ]:
%%time

bart_preds = []
for i in tqdm(range(0, len(dest), 50)):
    gc.collect()
    pred = restore(dest[i:i+50])
    if i == 0:
        print(len(pred))
    for dict_ in pred:
        bart_preds.append(dict_["translation_text"])

50

CPU times: user 3min 32s, sys: 44.3 s, total: 4min 16s
Wall time: 4min 16s


In [ ]:
bart_preds

In [ ]:
dest[:10]

In [ ]:
lines[:10]

## Punctuator

In [ ]:
!pip install punctuator

     |████████████████████████████████| 1.4MB 18.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.12.2-cp36-none-any.whl size=9693 sha256=b9f14c02779ea63b361dfcc95b2aa707ba316e56419e77d3cc970115888a7e11
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for punctuator: filename=punctuator-0.9.6-cp36-none-any.whl size=25154 sha256=326ad16331dfc951d2402e3c7e15d50d5059be61ee020f07d2ef33725d6d3987
  Stored in directory: /root/.cache/pip/wheels/9a/4b/83/024d41a639f2945705721ebc0b9c596880d492a487925f941a
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434673 sha256=694d5fa6215a2f9524d8f88d23d62db89b403b898d6076cc6e94102c86ffed1d
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully buil

In [ ]:
from punctuator import Punctuator

In [ ]:
#https://drive.google.com/drive/folders/0B7BsN5f2F1fZQnFsbzJ3TWxxMms

#first
!gdown https://drive.google.com/uc?id=0B7BsN5f2F1fZd1Q0aXlrUDhDbnM

#second
!gdown https://drive.google.com/uc?id=0B7BsN5f2F1fZZ2ZXd3R0dEh6NDA

Downloading...
From: https://drive.google.com/uc?id=0B7BsN5f2F1fZd1Q0aXlrUDhDbnM
To: /content/Demo-Europarl-EN.pcl
144MB [00:00, 159MB/s]
Downloading...
From: https://drive.google.com/uc?id=0B7BsN5f2F1fZZ2ZXd3R0dEh6NDA
To: /content/INTERSPEECH-T-BRNN.pcl
72.5MB [00:02, 32.6MB/s]


In [ ]:
#p = Punctuator('/content/Demo-Europarl-EN.pcl')

p = Punctuator("/content/INTERSPEECH-T-BRNN.pcl")

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [ ]:
%%time
print(p.punctuate('hello how are you'))

Hello, how are you.
CPU times: user 19.6 ms, sys: 16.4 ms, total: 36 ms
Wall time: 28 ms


In [ ]:
%%time

lstm_preds = []
for txt in tqdm(dest):
    pred = p.punctuate(txt)
    lstm_preds.append(pred)


CPU times: user 4min 16s, sys: 2min 51s, total: 7min 8s
Wall time: 3min 44s


In [ ]:
lstm_preds

['28 year old chef found dead at san francisco mall.',
 'A 28 year, old chef, who had recently moved to san francisco was found dead in the stairwell of a local mall. This week.',
 'But the victim\'s brother says, he can\'t think of anyone who would want to hurt him, saying, "things were finally going well for him ".',
 "The body found at the westfield mall wednesday morning was identified as 28 year old, san francisco, resident frank galicia. The san francisco medical examiner's office said,.",
 'The san francisco, police department said, the death was ruled a homicide and an investigation is ongoing.',
 "The victim's brother louis galicia told abc station kgo in san francisco that frank previously a line cook in boston had landed his dream job as line chef at san francisco's sons & daughters, restaurant, six months ago,.",
 'A spokesperson for sons & daughters said, they were "shocked and devastated" by his death.',
 '"We are a small team that operates like a close, knit family, and 

## Calculating

In [ ]:
MAPPING = {
    ",":1,
    ".":2,
    "!":3,
    "?":4,
    ";":5,
    ":":6,
}

In [ ]:
MAPPING.keys()

dict_keys([',', '.', '!', '?', ';', ':'])

In [ ]:
import re

def _split_txt(txt):
    """
    Split texts based on words and punctuations
    """
    return re.findall(r"[\w']+|[.,!?;:]", txt)

def assign_labels(txt):
    """
    assign labels to splitted text
    """
    mapped_ = [0] * len(txt)
    mapped = [MAPPING[tx] if tx in MAPPING.keys() else 0 for tx in txt]
    
    return mapped


"""
def _remove_other_labels(mapped):
    #For classification approach
    tgt_mapped = []
    for i in range(len(mapped)-1):
        if mapped[i]==0 and mapped[i+1]!=0:
            continue
        tgt_mapped.append(mapped[i])      
    return tgt_mapped

"""

'\ndef _remove_other_labels(mapped):\n    #For classification approach\n    tgt_mapped = []\n    for i in range(len(mapped)-1):\n        if mapped[i]==0 and mapped[i+1]!=0:\n            continue\n        tgt_mapped.append(mapped[i])      \n    return tgt_mapped\n\n'

In [ ]:
gold = [assign_labels(_split_txt(_txt)) for _txt in lines]
bart = [assign_labels(_split_txt(_txt)) for _txt in bart_preds]
lstm = [assign_labels(_split_txt(_txt)) for _txt in lstm_preds]

In [ ]:
pd.DataFrame({"input":dest, 
              "bart":bart_preds,
              "punctuator2":lstm_preds,
              "GT":lines}).to_csv("results.csv", header=True)

In [ ]:
assert len(gold) == len(bart)
assert len(gold) == len(lstm)

In [ ]:
print(f"{gold[5]}\n{bart[5]}\n{lstm[5]}")

[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2]


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
m = MultiLabelBinarizer().fit(gold)

In [ ]:
res = precision_recall_fscore_support(m.transform(gold), m.transform(bart), average=None)
bart_scores = pd.DataFrame(res, columns=list(MAPPING.keys())+["overall"])

In [ ]:
res = precision_recall_fscore_support(m.transform(gold), m.transform(lstm), average=None)

lstm_scores = pd.DataFrame(res, columns=list(MAPPING.keys())+["overall"])

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
list(MAPPING.keys())

[',', '.', '!', '?', ';', ':']

In [ ]:
bart_scores.T

,0,1,2,3
",",1.000000,1.000000,1.000000,3007.0
.,0.923992,0.808092,0.862165,1730.0
!,0.925158,0.970512,0.947293,2713.0
?,0.000000,0.000000,0.000000,30.0
;,0.787234,0.840909,0.813187,44.0
:,0.120000,0.103448,0.111111,29.0
overall,0.249110,0.534351,0.339806,131.0


In [ ]:
#second BRNN lstm model
lstm_scores.T

,0,1,2,3
",",1.000000,1.000000,1.000000,3007.0
.,0.746083,0.798266,0.771293,1730.0
!,0.902228,1.000000,0.948601,2713.0
?,0.000000,0.000000,0.000000,30.0
;,0.666667,0.045455,0.085106,44.0
:,0.000000,0.000000,0.000000,29.0
overall,0.000000,0.000000,0.000000,131.0


In [ ]:
#first Europarl-EN lstm model
lstm_scores.T

,0,1,2,3
",",1.000000,1.000000,1.000000,3007.0
.,0.746083,0.798266,0.771293,1730.0
!,0.902228,1.000000,0.948601,2713.0
?,0.000000,0.000000,0.000000,30.0
;,0.666667,0.045455,0.085106,44.0
:,0.000000,0.000000,0.000000,29.0
overall,0.000000,0.000000,0.000000,131.0
